In [51]:
#import dependencies
import numpy as np
import pandas as pd
import requests
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from scipy.stats import linregress
import scipy.stats as st
import os

In [52]:
# Read in csv and store into final data frame
fire_csv = "Fire Data.csv"


# pd.read_csv('file.csv', index_col=0)
fire_data_df = pd.read_csv(fire_csv)
fire_data_df.head()

,incident_name,incident_is_final,incident_date_last_update,incident_date_created,incident_administrative_unit,incident_administrative_unit_url,incident_county,incident_location,incident_acres_burned,incident_containment,...,incident_latitude,incident_type,incident_id,incident_url,incident_date_extinguished,incident_dateonly_extinguished,incident_dateonly_created,is_active,calfire_incident,notification_desired
0,Bridge Fire,True,2018-01-09T13:46:00Z,2017-10-31T11:22:00Z,Shasta-Trinity National Forest,NaN,Shasta,"I-5 and Turntable Bay, 7 miles NE of Shasta Lake",37.0,100.0,...,40.774000,NaN,2ca11d45-8139-4c16-8af0-880d99b21e82,https://www.fire.ca.gov/incidents/2017/10/31/b...,2018-01-09T13:46:00Z,1/9/2018,10/31/2017,N,False,False
1,Pala Fire,True,2020-09-16T14:07:35Z,2009-05-24T14:56:00Z,CAL FIRE San Diego Unit,NaN,San Diego,"Hwy 76 and Pala Temecula, northwest of Pala",122.0,100.0,...,1.000000,Wildfire,8f61f461-552d-4538-b186-35ab030da416,https://www.fire.ca.gov/incidents/2009/5/24/pa...,2009-05-25T00:00:00Z,5/25/2009,5/24/2009,N,True,False
2,River Fire,True,2013-02-28T20:00:00Z,2013-02-24T08:16:00Z,CAL FIRE San Bernardino Unit,NaN,Inyo,"south of Narrow Gauge Rd & north of Hwy 136, e...",406.0,100.0,...,36.602575,NaN,094719ba-a47b-4abb-9ec5-a506b2b9fd23,https://www.fire.ca.gov/incidents/2013/2/24/ri...,2013-02-28T20:00:00Z,2/28/2013,2/24/2013,N,True,False
3,Fawnskin Fire,True,2013-04-22T09:00:00Z,2013-04-20T17:30:00Z,San Bernardino National Forest,NaN,San Bernardino,"west of Delamar Mountain, north of the communi...",30.0,100.0,...,34.288877,NaN,58f89ff8-bd3e-4355-b1c0-8fa05c747d3f,https://www.fire.ca.gov/incidents/2013/4/20/fa...,2013-04-22T09:00:00Z,4/22/2013,4/20/2013,N,False,False
4,Gold Fire,True,2013-05-01T07:00:00Z,2013-04-30T12:59:00Z,CAL FIRE Madera-Mariposa-Merced Unit,NaN,Madera,Between Road 210 and Road 200 near Fine Gold C...,274.0,100.0,...,37.116295,NaN,357ffc13-bef9-48eb-810f-c5de851972eb,https://www.fire.ca.gov/incidents/2013/4/30/go...,2013-05-01T07:00:00Z,5/1/2013,4/30/2013,N,True,False


In [53]:
#drop the columns we don't need
fire_data_df.drop(['incident_is_final', 'incident_date_last_update', 'incident_administrative_unit', 'incident_date_created', 'incident_administrative_unit_url', 'incident_location', 'incident_containment', 'incident_containment', 'incident_control', 'incident_cooperating_agencies', 'incident_id', 'incident_url', 'incident_date_extinguished', 'is_active', 'calfire_incident', 'notification_desired'], axis =1, inplace=True)
fire_data_df.head()

,incident_name,incident_county,incident_acres_burned,incident_longitude,incident_latitude,incident_type,incident_dateonly_extinguished,incident_dateonly_created
0,Bridge Fire,Shasta,37.0,-122.309000,40.774000,NaN,1/9/2018,10/31/2017
1,Pala Fire,San Diego,122.0,1.000000,1.000000,Wildfire,5/25/2009,5/24/2009
2,River Fire,Inyo,406.0,-118.016510,36.602575,NaN,2/28/2013,2/24/2013
3,Fawnskin Fire,San Bernardino,30.0,-116.941311,34.288877,NaN,4/22/2013,4/20/2013
4,Gold Fire,Madera,274.0,-119.635004,37.116295,NaN,5/1/2013,4/30/2013


In [57]:
#rename columns
fire_data_df.rename(columns={'incident_county': 'county', 'incident_acres_burned':'acres_burned', 'incident_longitude':'longitude', \
                       'incident_latitude': 'latitude', 'incident_dateonly_extinguished':'date_extinguished', 'incident_dateonly_created':'date_created'}, inplace=True)
fire_data_df.head()

,incident_name,county,acres_burned,longitude,latitude,incident_type,date_extinguished,date_created
0,Bridge Fire,Shasta,37.0,-122.309000,40.774000,NaN,1/9/2018,10/31/2017
1,Pala Fire,San Diego,122.0,1.000000,1.000000,Wildfire,5/25/2009,5/24/2009
2,River Fire,Inyo,406.0,-118.016510,36.602575,NaN,2/28/2013,2/24/2013
3,Fawnskin Fire,San Bernardino,30.0,-116.941311,34.288877,NaN,4/22/2013,4/20/2013
4,Gold Fire,Madera,274.0,-119.635004,37.116295,NaN,5/1/2013,4/30/2013


In [58]:
#only use the wildfire incident type
clean_data_df = fire_data_df.loc[fire_data_df["incident_type"] =="Wildfire"]
clean_data_df


,incident_name,county,acres_burned,longitude,latitude,incident_type,date_extinguished,date_created
1,Pala Fire,San Diego,122.0,1.00000,1.00000,Wildfire,5/25/2009,5/24/2009
188,Sand Fire,"Amador, El Dorado",4240.0,-120.78190,38.56910,Wildfire,8/2/2014,7/25/2014
840,Atlas Fire (Southern LNU Complex),"Napa, Solano",51624.0,-122.24367,38.39206,Wildfire,2/9/2018,10/8/2017
841,Tubbs Fire (Central LNU Complex),"Napa, Sonoma",36807.0,-122.62879,38.60895,Wildfire,2/9/2018,10/8/2017
842,Nuns / Adobe / Norrbom/ Pressley / Partrick Fi...,"Napa, Sonoma",56556.0,-122.52090,38.40410,Wildfire,2/9/2018,10/8/2017
...,...,...,...,...,...,...,...,...
1729,Bonny Doone Complex (Previously Fanning Fire),Santa Cruz,20.0,-122.10509,37.08931,Wildfire,NaN,1/19/2021
1730,North Butano Fire,San Mateo,15.0,-122.28823,37.18259,Wildfire,NaN,1/18/2021
1733,72 Fire,Riverside,43.0,-116.10723,33.52871,Wildfire,1/20/2021,1/19/2021
1735,Panther Ridge Fire,Santa Cruz,20.0,-122.13372,37.18381,Wildfire,1/22/2021,1/19/2021


In [59]:
#drop rows with missing information
clean_data_df = clean_data_df.dropna(how='any')
clean_data_df

,incident_name,county,acres_burned,longitude,latitude,incident_type,date_extinguished,date_created
1,Pala Fire,San Diego,122.0,1.00000,1.00000,Wildfire,5/25/2009,5/24/2009
188,Sand Fire,"Amador, El Dorado",4240.0,-120.78190,38.56910,Wildfire,8/2/2014,7/25/2014
840,Atlas Fire (Southern LNU Complex),"Napa, Solano",51624.0,-122.24367,38.39206,Wildfire,2/9/2018,10/8/2017
841,Tubbs Fire (Central LNU Complex),"Napa, Sonoma",36807.0,-122.62879,38.60895,Wildfire,2/9/2018,10/8/2017
842,Nuns / Adobe / Norrbom/ Pressley / Partrick Fi...,"Napa, Sonoma",56556.0,-122.52090,38.40410,Wildfire,2/9/2018,10/8/2017
...,...,...,...,...,...,...,...,...
1726,Sanderson Fire,Riverside,1933.0,-117.03459,33.93244,Wildfire,12/14/2020,12/13/2020
1727,Erbes Fire,Ventura,250.0,-118.83967,34.20955,Wildfire,1/15/2021,1/14/2021
1733,72 Fire,Riverside,43.0,-116.10723,33.52871,Wildfire,1/20/2021,1/19/2021
1735,Panther Ridge Fire,Santa Cruz,20.0,-122.13372,37.18381,Wildfire,1/22/2021,1/19/2021


In [60]:
#Connect to local database
rds_connection_string = "postgres:postgres@localhost:5432/fire_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [61]:
#Use pandas to load csv converted DataFrame into database
clean_data_df.to_sql(name= 'clean_data_df', con=engine, if_exists='append', index=False)

DataError: (psycopg2.errors.InvalidTextRepresentation) invalid input syntax for type integer: "5/25/2009"
LINE 1: ... Fire', 'San Diego', 122.0, 1.0, 1.0, 'Wildfire', '5/25/2009...
                                                             ^

[SQL: INSERT INTO clean_data_df (incident_name, county, acres_burned, longitude, latitude, incident_type, date_extinguished, date_created) VALUES (%(incident_name)s, %(county)s, %(acres_burned)s, %(longitude)s, %(latitude)s, %(incident_type)s, %(date_extinguished)s, %(date_created)s)]
[parameters: ({'incident_name': 'Pala Fire', 'county': 'San Diego', 'acres_burned': 122.0, 'longitude': 1.0, 'latitude': 1.0, 'incident_type': 'Wildfire', 'date_extinguished': '5/25/2009', 'date_created': '5/24/2009'}, {'incident_name': 'Sand Fire', 'county': 'Amador, El Dorado', 'acres_burned': 4240.0, 'longitude': -120.7819, 'latitude': 38.5691, 'incident_type': 'Wildfire', 'date_extinguished': '8/2/2014', 'date_created': '7/25/2014'}, {'incident_name': 'Atlas Fi

In [46]:
#Print table name created in SQL
engine.table_names()

['new_combine_df', 'clean_data_df']

In [ ]:
#Confirm data has been added by querying the customer_name table
pd.read_sql_query('select * from fire_db', con=engine).head()